# Napredni operacijski sustavi

### 2. laboratorijska vježba - Digitalni potpis

Zagreb, *04.05.2020.*

## Izjava

Tekstovi zadataka se koriste samo u edukativne svrhe, te njihova prava još uvijek pripadaju autorima. Tekstovi zadatka preuzeti su sa sljedećih poveznica: [generalno](http://www.zemris.fer.hr/predmeti/os2/), [vježba 2](http://www.zemris.fer.hr/predmeti/os2/kriptografija/digitalni_potpis.html). Također, bilo kakve izmjene su isključivo radi estetike, i ne mijenjaju intelektualnog vlasnika na mene ili bilo kog tko uređuje ovu datoteku.

## Sadržaj

- [Priprema](#Priprema)
- [Simetrična enkripcija](#Simetrična-enkripcija)
- [Asimetrična enkripcija](#Asimetrična-enkripcija)
- [Funkcije sažimanja](#Funkcije-sažimanja)
- [Digitalne omotnice](#Digitalne-omotnice)
- [Digitalni potpisi](#Digitalni-potpisi)
- [Digitalni pečati](#Digitalni-pečati)

## Priprema

Kako bi uopće mogli pokrenuti ovu vježbu, potreban nam je modul za enkripciju **Cryptodome** (`pip install pycryptodome`). Koristi se **Python 3.6.10**, iako vjerojatno vježba radi na bilo kojem Pythonu verzije **3.6** ili više. Ukoliko imate problem s pokretanjem (Jupyter ne pronalazi `Crypto`), napravite ovo:

```
pip3 install ipykernel --upgrade
python3 -m ipykernel install --user
```

Prvo ćemo učitati par modula koji će nam olakšavati stvari:

In [1]:
from base64 import b64decode, b64encode
import json
from sys import stderr

In [2]:
from IPython.display import display, Markdown, Code

Onda ćemo učitati 5 metoda koje smo implementirali (tj. obuhvatili klasama radi lakšeg korištenja):

In [3]:
from symmetric import AES, DES3
from asymmetric import RSA
from hashes import SHA2, SHA3

Zatim ćemo učitati sve komponente:

In [4]:
from components import Envelope, Signature, Seal

Sve primjere izvodit ćemo na poruci "Ana voli Milovana":

In [5]:
message = "Ana voli Milovana"

## Simetrična enkripcija

Na raspolaganju imamo dva algoritma:

- AES
- Triple DES

Dok **Triple DES** ima jednu veličinu ključa ($64$ bitova), moguće je odabrati $3$ veličine ključa za **AES**:

- $128$ bita
- $192$ bita
- $256$ bita

Također, oba algoritma koriste jedan $5$ načina kriptiranja (zadani je **CBC**):

- ECB
- CBC
- OFB
- CFB
- CTR

### AES

Možemo demonstrirati rad ovog algoritma za različite ključeve:

In [6]:
aes_ = [AES(x) for x in (128, 192, 256)]

Osim ovakve inicijalizacije, moguće je učitati i veličinu ključa u broju bajtova. Enkriptirajmo podatke koristeći svaki od ovih načina rada:

In [7]:
aes_crypts = [x.encrypt(message) for x in aes_]
print("\n\n".join([x.decode("utf8") for x in aes_crypts]))

5DhyVVf+th5Q8Q7rN7Ybv/nce8CDiNPjWeTTEC+pmeQ=

0I0DLn1LfdmeC53Ndw2B5VJJUhCTbWcmOJcCx0Mco5g=

RMID4dZqb5WEF/W6ms+eaY809Wco10g49ewdaqr37Yo=


Možemo ih pokušati i dekriptirati:

In [8]:
aes_decrypts = [x.decrypt(y) for x, y in zip(aes_, aes_crypts)]
print("\n\n".join([x.decode("utf8") for x in aes_decrypts]))

Ana voli Milovana

Ana voli Milovana

Ana voli Milovana


Kako imamo nekoliko načina kriptiranja, bilo bi dobro da prikažemo generirane podatke za svaki od tih tipova, na primjer na $128$-bitnim ključevima:

In [9]:
aes_modes = ("ecb", "cbc", "ofb", "cfb", "ctr")

aes_m = [AES(128, x) for x in aes_modes]
aes_dicts = [x.save_to_dict() for x in aes_m]
aes_display_pairs = [(title, code) for title, code in zip([x.upper() for x in aes_modes],
                                                          [json.dumps(x, indent=2) for x in aes_dicts])]

In [10]:
for title, code in aes_display_pairs:
    display(Markdown(f"#### {title} način rada"))
    display(Code(code, language="json"))

#### ECB način rada

{
  "desc": "AES key",
  "key_length": 128,
  "cipher_mode": "ecb",
  "secret_key": "5b1a1cfe75846271f478f25309e846e6"
}

#### CBC način rada

{
  "desc": "AES key",
  "key_length": 128,
  "cipher_mode": "cbc",
  "init_vector": "d23869bc93221170407a7c08a54843cb",
  "secret_key": "aa280ab5f664e130d85f8a7473939259"
}

#### OFB način rada

{
  "desc": "AES key",
  "key_length": 128,
  "cipher_mode": "ofb",
  "init_vector": "771fb2f4d06065d7f7081f69cd4abbb7",
  "secret_key": "c098747386cb648065331cba9c6e1971"
}

#### CFB način rada

{
  "desc": "AES key",
  "key_length": 128,
  "cipher_mode": "cfb",
  "init_vector": "310098fc9229d939890091f7de5e7c77",
  "secret_key": "4c2e1534c96b160a565bc74890b911fc"
}

#### CTR način rada

{
  "desc": "AES key",
  "key_length": 128,
  "cipher_mode": "ctr",
  "secret_key": "806d814f67d7b2ec451f63e7607e6ed7"
}

### Triple DES

Kako se ovdje radi o jednoj veličini ključa, demonstrirat ćemo ponašanje za različite načine kriptiranja:

In [11]:
des_modes = ("ecb", "cbc", "ofb", "cfb", "ctr")

des_ = [DES3(x) for x in des_modes]

In [12]:
des_crypts = [x.encrypt(message) for x in des_]
print("\n\n".join([x.decode("utf8") for x in des_crypts]))

Xcce3admnhjUp0aiJ2RCrJdivpybvV5b

m5NaEalbXAjAPCkaGaz4qgbVE8hcLu92

c6En2TUq/uf/ci6eTNafEurD3DM/lnDC

kFhXp4zePqBelAAmQSC6PhB4t0vaFxZc

dLoSuerayc21LdjyYRkZdnJVbSPMLqLX


Kao i prije, ovo možemo dekriptirati:

In [13]:
des_decrypts = [x.decrypt(y) for x, y in zip(des_, des_crypts)]
print("\n\n".join([x.decode("utf8") for x in des_decrypts]))

Ana voli Milovana

Ana voli Milovana

Ana voli Milovana

Ana voli Milovana

Ana voli Milovana


Konačno, usporedit ćemo dobivene ključeve:

In [14]:
des_dicts = [x.save_to_dict() for x in des_]
des_display_pairs = [(title, code) for title, code in zip([x.upper() for x in des_modes],
                                                          [json.dumps(x, indent=2) for x in des_dicts])]

In [15]:
for title, code in des_display_pairs:
    display(Markdown(f"#### {title} način rada"))
    display(Code(code, language="json"))

#### ECB način rada

{
  "desc": "Triple DES key",
  "key_length": 64,
  "cipher_mode": "ecb",
  "secret_key": "d39ebf75ba08bc34c491c4548c8cd570108af292e6d0b0f8"
}

#### CBC način rada

{
  "desc": "Triple DES key",
  "key_length": 64,
  "cipher_mode": "cbc",
  "init_vector": "4850e6f02b2e4aae",
  "secret_key": "3d5d5bd6bc76ad6b19cd7c266b8aab312a62e094fd62b380"
}

#### OFB način rada

{
  "desc": "Triple DES key",
  "key_length": 64,
  "cipher_mode": "ofb",
  "init_vector": "24b26c88133cdf17",
  "secret_key": "40ce9b6794b368430d574f02e62ca1d5e37901f425073725"
}

#### CFB način rada

{
  "desc": "Triple DES key",
  "key_length": 64,
  "cipher_mode": "cfb",
  "init_vector": "c2a30e656d7f6c26",
  "secret_key": "2a4302109b349892268392373432a7c7aefd619bba2a1380"
}

#### CTR način rada

{
  "desc": "Triple DES key",
  "key_length": 64,
  "cipher_mode": "ctr",
  "secret_key": "a4f1a119bfdc62682c64c185a897384f02947907859d9419"
}

## Asimetrična enkripcija

Od ovih algoritama, implementirali smo samo **RSA**. Dostupni su nam ključevi veličina $1024$, $2048$ i $3072$ bita, kako definira FIPS, a zadana je veličina ključa od $1024$ bita. Veličina javnog eksponenta je proizvoljna (zadano je $65537$).

### RSA

Pogledajmo algoritam po ključevima:

In [16]:
rsa_key_lengths = (1024, 2048, 3072)

rsa_ = [RSA(x) for x in rsa_key_lengths]

Kao i prije, možemo koristeći ključeve enkriptirati našu poruku:

In [17]:
rsa_crypts = [x.encrypt(message) for x in rsa_]
print("\n\n".join([b64encode(x).decode("utf8") for x in rsa_crypts]))

g5W/07BysDu3AtYV2sly/7dHWjDilri3SBRhzas5BoDiIrMH+MIIeDRLSrfZM/rLzKM7PDpX6I7YD0PFalLrji+Kzrr2HPm+Ybe/6Ph5jHsksLGw4cuTUUPq4TqIMbTWoEyYp+0Dj8P72wQgztRnVwyiVj2B4btiXxhhw34qVwQ=

EQp+LeiRbfwCA/EfoAfwxuR0j2DwrEJodO9V3MjujO7u84FCGF3uNyKDPMCrbPG/I74XeX84O9t2HBoU3DYq5J/NG6JJQaorb9T1t4dTsO3Jljd9re5zt9hgKuueqcRNUphxSuOQ7wfFF0fBnZPAUGRQqZdVXO4OEGpQ04WMfZ01nCp3iR6/ulmwE6h1z17gWDFJqi6pENyVaXXGGH3Ji2Q/r5lEa6J1NabgA3LwLmJ9FaDS8lHlapmfpi15upD5ZNf/uLEbCatB1zueOEz+lpGANLxppegkMp+rkhsDMQqMIlYfYDijmDyxQwsVqmsFzESFaiMfBRfSbX2eguLLCg==

a/NX/DBLHXsuBjpZuAENg/59ySA3sqz77x48J/71MgYSoPlVPkaDOg2WmGD6qD3WGG9hqZFHgA5i2an3NmwG/h9g1tJfrYBUbodRzzuJVHwyBcOitpTWOAsFdzCwqP6zFu+YDeOC8h2CFuj7EP+P8BRzYGzLkE71Z9RE6KHq9noqLNd7g8X/vK29Uog5lKe2QveyvkDtuOZ4pKRxFRXfoZ7oMg5KSi0JYbbBWsHi981m3dYk2n5E97hTdXXwyRKkcWli2+JLRVGTPftmVzF0TLmxi30GPHblgNU9gHVWjvxqLhtqJ0MLAcXEELSPG85ONJqyTenaB5PJOZVO1dGlgETdjT3XGqY54erqICDekn9bR9E8WDn23PNRHbB85frFszYwt2v8QKbHNL8GCxZmxIAsxgklTCmgoQNd3xLl6WIrOUzO/IFBkZ8LgGJlFNpPyWwx8gQwJAspBWVYDIQIcngl0JpLHQ5o

Kao i prije, možemo ih dekriptirati:

In [18]:
rsa_decrypts = [x.decrypt(y) for x, y in zip(rsa_, rsa_crypts)]
print("\n\n".join([x.decode("utf8") for x in rsa_decrypts]))

Ana voli Milovana

Ana voli Milovana

Ana voli Milovana


Naposljetku, možemo pogledati kako izgledaju zapisi ovih ključeva:

In [19]:
rsa_dicts = [x.save_full_to_dict() for x in rsa_]
rsa_display_pairs = [(title, code) for title, code in zip([str(x) for x in rsa_key_lengths],
                                                          [json.dumps(x, indent=2) for x in rsa_dicts])]

In [20]:
for title, code in rsa_display_pairs:
    display(Markdown(f"#### {title}-bitni ključ"))
    display(Code(code, language="json"))

#### 1024-bitni ključ

{
  "desc": "RSA key",
  "mod": 123711025032605999004910143755050664136011393102847659603218178198525449361780304061125593005473438677424583768183891135547919800537584313182536890926622152949935256435643071277854864952267992908858235864163551719063842584293488854668012508817740097530856818070457372499555236474441978308520955898785666985217,
  "pub_exp": 65537,
  "priv_exp": 10689771194281822060283597724718127332685849507017811256740841709847103464237939818700188186673117219742365654198778941675814728023015090186500853156498788869876493467615952333746455541339012539127941105578913588163809865075400138295976995682584271395307665330305170453348310269277030476732322040754576538237
}

#### 2048-bitni ključ

{
  "desc": "RSA key",
  "mod": 24010225806991601999819732485821150160118766235184884368188820272371469041332151964271201128042263326107138899220531601684339964070300594116687388276532889075581684953752561045520361511108050549190010787116244468159348785990560697531192595019149398299207173637428515827925523797098950739036206493868406070591123945723613831255387450225301478090549740668497179610539696997841010556024521041151539287266417130193772079453963432452934869726650350891863345954013834427181020756556721399526536479859762155929317831234428533241151227334429788794333160751913604257300563083464254266697063049355678453650429133313303595196901,
  "pub_exp": 65537,
  "priv_exp": 551312817724662339483478453938180378503014430366267200106751042004877636485720534673555078263916046333677559040598389285461916005693333700249656501286335595321286788245397952505209128898674510603772442530866969048046040630251543753933243664972712404808331097375889623190817284800968476389311657112617906453261655683703940920869821182992666892487147707768671671009337369877737689732923282785676978804551545739132669446142277744876525849683315828873743086120183855695741319859335144742961494008383015332263384275886298434782581222394515691748739920802253425864432154877034918949137858649220984224541319625511610174001
}

#### 3072-bitni ključ

{
  "desc": "RSA key",
  "mod": 4304264773448268311502553943505793628975473934811545035537069544838842370987019670456683570648658830000185260296694079941413627175068849791392062457760526293228499914486564419318700210046415209803690978099827482993067198255641703357245070307080846265341443784083897453678879076893363634168725791717443490669919555773319137335364841463567139685405343556052987690660019147123740887033474880543724780211508032939519692118628739011050921841903792872666745586868623706413643727302831665068509016395008332221399772850421098418474487647752358455217524431993922024676763297276054722156346731144851068048200183027125096093371922209012878763249667556375608529695075336511818573922522075087374122997810303388247131683095049026096404038507895580502055897528228665755883540016197605511468983177946303380288261504546743106070069518222175604069189865533696371792383528806654121380509266838642321778107681283334448178238943048121366423236291,
  "pub_exp": 65537,
  "priv_exp": 1381381524633984274468364696183184420987246033094148751582330343723322269694523471149357241580378124897293079936835170718948881095766103386824988169645805415650320257280557691556376116055148253777271348129662197686698847687732303076322315085658962715701460047158652595377082619349346435104914927097563039798288264897389587798872830775031015289121116178867853122627709274477831485679464686550744820516481505970931194353283157111546668890867181523273870636870893729182092232891059507757978201750480705578616428136915872339153196580442798276486088276274221947275462932287770822643276634789688403873847921881044470727361852290274115616660365925722132868624758852474763879245890250480442066907243289950164373036296595031271287464604187805351632740533284583442575960027460819919701875278567604038274849903001630575709983700448195548713496898181855318877492317036539846737374284188435554775729680679691775816019091649568689684019129
}

**Dodatno**: Dostupno je i spremanje samo privatnog ključa, tj. zapis bez `priv_exp` kroz metode `save_public_to_dict()` i `save_public_to_file()`.

## Funkcije sažimanja

U sklopu ove vježbe, implementirana su 2 algoritma za sažetke: **SHA-2** i **SHA-3**.

Svaki od njih može raditi s 4 duljine:

- $224$ bita
- $256$ bita
- $384$ bita
- $512$ bita

### SHA-2

Kao i prije, možemo napraviti instance sa sve $4$ duljine:

In [21]:
sha2_lengths = (224, 256, 384, 512)

sha2_ = [SHA2(x) for x in sha2_lengths]

Sad možemo sažeti našu poruku na sva $4$ načina:

In [22]:
print("\n\n".join(x.hash_now(message).hex() for x in sha2_))

d71d3ceb381e4a2139d5fee16f47ce1b2a01093a4cfd935956f74a7d

301a86f512742aefa145d718fb11a713c98ae62579b862fea2c94046f35a678e

4cf14d16378565669bfd2a65f0b91a434c5d07b5b83252d5741d870093727adf687dc5513a31bb6e2ae4ddde6d77db87

d11a7f5a49f77205dd6493a609b3ca0d44e119153b53fc58e16f455ec0d8839fe5b0b2a81a834ac6f6c3c9123094a5ca746fbc1adb77fc1f576f49ef2579e7fd


### SHA-3

Ponovit ćemo postupak kao i za prethodni algoritam:

In [23]:
sha3_lengths = (224, 256, 384, 512)

sha3_ = [SHA3(x) for x in sha3_lengths]

In [24]:
print("\n\n".join(x.hash_now(message).hex() for x in sha3_))

c1d3d11a552870ec03c618c0f4bdbd7e7d4113eaf96d3aa8a5094680

2564cc6e6a127fb50ea460394a08fc917f1d041e8c0d29841ba084f21da3cb77

b63b3a4e4f4f9626a8f533b9e6370f4e848c4e855acb57b3abc4fa4964d61a593fad4b996534858fe698325f98d5ef27

e1a53a38adf6a9d332746e15b523496fbd3fed9cdfdc064841e1b8d6ee1d5048902a3a8a3350f954777cbcb9ed1ba8924f3304bbf28fb699a7e6d97c2e714ed1


## Digitalne omotnice

Digitalna omotnica predstavlja neki dokument čiji sadržaj enkriptiramo nekom simetričnom enkripcijom, a potom tajni ključ te enkripcije enkriptiramo javnim ključem primatelja. Zatim šaljemo sljedeće podatke:

- metoda kriptiranja podataka
- inicijalizacijski vektor za ECB, CBC i OFB načine rada
- enkriptirani podaci
- enkriptirani tajni ključ

Radi reprezentativnosti, mi šaljemo nešto više podataka (iako je to nepotrebno).

Za simetričnu enkripciju možemo birati algoritme **AES** i **Triple DES**. Za asimetričnu enkripciju imamo jedino **RSA**. Pa omotajmo našu poruku u digitalnu omotnicu:

In [25]:
env_algorithms = ("AES", "Triple DES")

env_ = [Envelope(x(), RSA()) for x in (AES, DES3)]

In [26]:
env_dicts = [x.envelop_to_dict(message) for x in env_]
env_display_pairs = [(title, code) for title, code in zip([str(x) for x in env_algorithms],
                                                          [json.dumps(x, indent=2) for x in env_dicts])]

In [27]:
for title, code in env_display_pairs:
    display(Markdown(f"#### Algoritam {title}"))
    display(Code(code, language="json"))

#### Algoritam AES

{
  "desc": "Envelope",
  "method": {
    "data": "aes",
    "secret_key": "rsa"
  },
  "cipher_mode": "cbc",
  "key_length": {
    "data": 128,
    "secret_key": 1024
  },
  "init_vector": "589c42a8390de6efcb03ff87176b5cc8",
  "encrypted_data": "ZgsBgu7dLlw6a3MHomEG0GcUHqBN3yWGjmK/1jezUOg=",
  "encrypted_key": "7378a358f77a2428e5e2aa8eaa73adc24fe2f952959d5b8b1a292decbadd24cd2a1bf998757870c11f89ef44ee2f90f55ed808b43b1ccc2a749d830439f81124ead0cd103c372473eedd539b9409b822785e8213e1f1eb058b349e725905776dbed33ca8ffb53c12bf8b99c915406fb40e23050dfaaab264333aaac43d5a7e7f"
}

#### Algoritam Triple DES

{
  "desc": "Envelope",
  "method": {
    "data": "3des",
    "secret_key": "rsa"
  },
  "cipher_mode": "cbc",
  "key_length": {
    "data": 64,
    "secret_key": 1024
  },
  "init_vector": "24b79d57843bb6a6",
  "encrypted_data": "0PKDnklc8kxai4kSGCY25MTV1IyuxDFd",
  "encrypted_key": "84b2322149b104170a2e90b59ebc4596bc307425b612a9d7897974d3b4b53c21a7bf1783025041984bee886a3537ef1b9d7abaf311b94548604599e54d606d909b2046b75d6661c78d045a7bb8cd5d16cca1d69e0b4baa962cd927203f32b5ca551f7c5a045efb5a20fec9c795e0d17471af3619edaee3427e24be47a0e99811"
}

Možemo i pročitati ove omotnice:

In [28]:
print("\n\n".join([Envelope.open_from_dict(x, y.secret_key_cipher).decode("utf8")
                   for x, y in zip(env_dicts, env_)]))

Ana voli Milovana

Ana voli Milovana


## Digitalni potpisi

Digitalni potpisi su poruke sažete nekim algoritmom sažimanja, a potom enkriptiranje javnim ključem primatelja. U našem konkretnom slučaju to znači da ćemo poruku prvo sažeti **SHA-2** ili **SHA-3**, a potom je enkriptirati **RSA** ključem. Kako primatelj mora usporediti sažetak poruke nakon što dekriptira potpis, uz potpis mu šaljemo i poruku.

Ovo ćemo istestirati na oba algoritma sažimanja, za, recimo, $512$-bitne sažetke (i zadani $1024$-bitni **RSA** ključ):

In [29]:
sig_classes = (SHA2, SHA3)
sig_names = ("SHA-2", "SHA-3")

sig_ = [Signature(x(512), RSA()) for x in sig_classes]

In [30]:
sig_dicts = [x.sign_to_dict(message) for x in sig_]
sig_display_pairs = [(title, code) for title, code in zip([str(x) for x in sig_names],
                                                          [json.dumps(x, indent=2) for x in sig_dicts])]

In [31]:
for title, code in sig_display_pairs:
    display(Markdown(f"#### Uz {title}-512 sažetak"))
    display(Code(code, language="json"))

#### Uz SHA-2-512 sažetak

{
  "desc": "Signature",
  "method": {
    "hash": "sha2",
    "signature": "rsa"
  },
  "key_length": {
    "hash": 512,
    "signature": 1024
  },
  "data": "416e6120766f6c69204d696c6f76616e61",
  "signature": "c08dfb1f721e5b94fce98c5e6f5e46fc8e9e7182364911f708644cc3a92c1c7d17f28f1d81a63e76bee5a7547883f0e79e48cce3dc6092b239245bea9be040cea13d4f1860e76df7beb5a7e561da6de06fe4fd5e2287d3613e1954732225002c3e376ed70cfb94247968b020357b6ce404d88c5ed6d17199659f01fed1b6a3e0"
}

#### Uz SHA-3-512 sažetak

{
  "desc": "Signature",
  "method": {
    "hash": "sha3",
    "signature": "rsa"
  },
  "key_length": {
    "hash": 512,
    "signature": 1024
  },
  "data": "416e6120766f6c69204d696c6f76616e61",
  "signature": "91cbee202f64a1be79790d0c2bce5453fd0e3fb936544cbf8bbb13383d170fa476e55e0d3cc95d9fd7711adca6e2f1764ffda3fe2f6fe8104de2b3b018b15bb4e08e2f289cde100f11e19d0c5bf06a812c382e1cd3699c8487e6b3af92ee2597ec2da47dd2cf29e772e81bb88a970865a3ac2233fce9507256286981b5ff1957"
}

Kao i prije, možemo provjeriti odgovaraju li potpisi za podatke:

In [32]:
sig_valids = [x.self_verify(message, y["signature"])
              for x, y in zip(sig_, [Signature.read(z) for z in sig_dicts])]
print("\n\n".join("Potpis je ispravan." if x else "Potpis je neispravan." for x in sig_valids))

Potpis je ispravan.

Potpis je ispravan.


## Digitalni pečati

Digitalni pečati kombinacija su digitalnih omotnica i potpisa:

- prvo načinimo digitalnu omotnicu od poruke koristeći primateljev javni ključ
- zatim računamo potpis za tu digitalnu omotnicu (u našem slučaju računamo potpis konkatenirane enkriptirane poruke i enkriptiranog ključa) koristeći primateljev javni ključ (ne nužno isti kao u koraku prije)
- šaljemo trojku **enkriptirana poruka**, **enkriptiran ključ**, **digitalni potpis** primatelju koji će ih uz svoje privatne ključeve dekriptirati

Pečaćenje vršimo uz dosad sve dostupne algoritme. Ponovo radi reprezentativnosti nakon pečaćenja poruke šaljemo nešto više informacija.

Za pečat koristit ćemo **AES** enkripciju s $256$-bitnim ključevima, različite $1024$-bitne **RSA** ključeve, te **SHA-3-384** sažetke.

In [33]:
seal = Seal(message_cipher=AES(256), secret_key_cipher=RSA(1024),
            hash_cipher=SHA3(384), signature_cipher=RSA(1024))

Sada možemo zapečatiti našu poruku:

In [34]:
sealed_message = seal.seal_to_dict(message)
display(Code(json.dumps(sealed_message, indent=2), language="json"))

{
  "desc": "Seal",
  "method": {
    "message": "aes",
    "secret_key": "rsa",
    "hash": "sha3",
    "signature": "rsa"
  },
  "key_length": {
    "message": 256,
    "secret_key": 1024,
    "hash": 384,
    "signature": 1024
  },
  "cipher_mode": "cbc",
  "init_vector": "6dee510308dc7691def8d101bf846f88",
  "encrypted_data": "IbnWLYf8bYfi29mYgtQyod6CndAduYg8QZ70Jjx+WmQ=",
  "encrypted_key": "0d906d1d93e2dc8e0acc6f5a926b05403a7af49ca76abd9b8b4820259aa32987b1a7f313f93a08c56a65ccf52e71df4e52c3c6897febe19d148e02b061416beb3a6a2492a208f6bfb7c58c2c5bda98a8e0e39578c5d444e8b355aa7ffee060e23be8666abceaf2c2bf1531e1e38486a4ddcab26db9e02138c4e071539119b06a",
  "signature": "019b549f6c854cdf6384edaa808736a0297de2a2b15b3abbc33c097911805275308237c47d73862900b524480061cd1c03be8f267cbb708e9e984eeee3f20ae729bd473fd08f865a3040d06e1f390e108f560b657dff18f3a3d3170bf695759abb7a2e4f82107b73feded0194a61bee1ecca5a30cb1e65cbfd45c9968247fbc5"
}

Mogli bismo pokušati izmijeniti potpis tako da mu izmijenimo zadnji bajt u $1$, ili ako je već $1$, da ga promijenimo u $0$:

In [35]:
tampered_message_1 = dict(sealed_message)
tampered_message_1["signature"] = tampered_message_1["signature"][:-1] + \
                                  ("1" if tampered_message_1["signature"][-1] != "1" else "0")

Sad kad pokušamo otvoriti poruku, susrest će nas poruka da potpis ne valja:

In [36]:
try:
    print(Seal.open_from_dict(tampered_message_1,
                              seal.envelope.secret_key_cipher,
                              seal.signature.signature_cipher).decode("utf8"))
except ValueError as v:
    print(v, file=stderr)

Signature check failed!


Isto će se dogoditi ako nešto promijenimo u podacima:

In [37]:
tampered_message_2 = dict(sealed_message)
tampered_message_2["encrypted_data"] = tampered_message_2["encrypted_data"][:-1] + \
                                       ("1" if tampered_message_2["encrypted_data"][-1] != "1" else "0")

In [38]:
try:
    print(Seal.open_from_dict(tampered_message_2,
                              seal.envelope.secret_key_cipher,
                              seal.signature.signature_cipher).decode("utf8"))
except ValueError as v:
    print(v, file=stderr)

Signature check failed!


No ako pokušamo otvoriti originalnu poruku, nećemo imati problema:

In [39]:
try:
    print(Seal.open_from_dict(sealed_message,
                              seal.envelope.secret_key_cipher,
                              seal.signature.signature_cipher).decode("utf8"))
except ValueError as v:
    print(v, file=stderr)

Ana voli Milovana


## Konačni komentari

Ovi svi primjeri su izvedeni na instanci klasa kojima su stvarani entiteti. Naravno da to nije uvjet da stvari rade, nego je najjednostavniji način za prikazati situaciju. Svi entiteti imaju metode s kojima je moguće to što ispisuju spremiti u `.json` datoteku. U skladu s time moguće je učitavati javne ključeve umjesto cijelih ključeva u omotnice, potpise i pečate, a onda ih čitati koristeći privatne ključeve.

Međutim, to je već mehanizam serijalizacije pa nisam mislio da je relevantno za ovu vježbu. Svi kodovi su dostupni u `symmetric.py`, `asymmetric.py`, `hashes.py` i `components.py`, stoga je moguće provjeriti navedenu funkcionalnost, i još neku koju možda nisam pokrio.